In [1]:
#General imports
import numpy as np
import signal
import os
import cubeit.DMs as dms
import datetime

# Import cubeit functions
from cubeit.gates_DMs import(
    x, y, z, had, s, t, cnot, cnot_10, phase, rotation_x, rotation_y, rotation_z, swap, cz, cphase
)
from cubeit.visualisation import plot_bloch_sphere, plot_circuit, simulate_measurements
from cubeit.notebook_functions import plot_measure_DMs, get_gates_targets, gates_lookup


#Import plotting and visualisation libraries
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual, Text, SelectMultiple, Layout, VBox, HBox, Button, Dropdown, Output, Label, HTML, GridBox, FloatSlider, get_ipython
from IPython.display import display

In [3]:
state = None
last_bloch_fig = None
last_circuit_fig = None
last_measurement_fig = None

# Widgets
title = HTML("<h2 style='color:blue;'>Cub<i>e</i><sup>it</sup> Quantum Circuit Simulator</h2>")
init_label = HTML("<b>&larr;Initialise 2-qubit system in |00>, or enter [a,b,c,d] for [a|00>+b|01>+c|10>+d|11>]</b>")
noise_label = HTML("<b>&larr;Set noise type and level before applying gates </b>")
reinit_btn = Button(description="Initialise")
gate_label = HTML("<b>&larr; Enter gates and target qubits, e.g. Had(0);CNOT(0,1)Rotation_x(pi)(0).</b> <br> From: X,Y,Z,Had,S,T,CNOT,SWAP,CZ,Rotation_x,Rotation_y,Rotation_z,Phase,Cphase")
text_in = Text(description="Gates:")
state_in = Text(description="State:")
send_btn = Button(description="Apply gates")
noise_dropdown = Dropdown(options=["None", "Depolarising", "Dephasing", "Amplitude damping", "Bit flip"], description="Noise:")
noise_slider = FloatSlider(value=0.0, min=0.0, max=1.0, step=0.01, description="")
out = Output()
out1 = Output()
out2 = Output()
save_label = HTML("<b>&larr; Enter filename prefix to save all figures</b>")
filename_box = Text(description="Filename:")
save_button = Button(description="Save Figures")
exit_btn = Button(description="Exit", button_style='danger')

# Handlers
def on_initialise(_):
    out.clear_output()
    out1.clear_output()
    out2.clear_output()
    with out:
        global rho
        if state_in:
            state_in_ = state_in.value
            if len(state_in_.strip()) <1:
                init_state = [1.0,0.,0.,0.]
            else:
                init_state_unnormalised = [complex(x) for x in state_in_.replace('[','').replace(']','').split(',')]
                norm = sum([abs(x)**2 for x in init_state_unnormalised])
                init_state = [x/np.sqrt(norm) for x in init_state_unnormalised]
                if not len(init_state) ==4:
                    print("Invalid initial state, must be 4 complex numbers separated by commas.")
                    init_state = [1.0,0.,0.,0.]
        else:
            init_state = [1.0,0.,0.,0.]

        state = np.array(init_state, dtype=complex)
        rho_ = np.outer(state, state)
        rho = dms.DensityMatrix2Qubit(rho_)
        global last_bloch_fig
        last_bloch_fig = plot_bloch_sphere(rho)
    with out2:
        global last_measurement_fig
        last_measurement_fig = plot_measure_DMs(rho, meas_shots=1000)

def on_send(_):
    out.clear_output()
    out1.clear_output()
    out2.clear_output()
    noise_dict = {'depolarising':0.0, 'dephasing':0.0, 'amplitude damping':0.0, 'bit flip':0.0} 
    with out:
        gates, targets = get_gates_targets(text_in.value)
        for gate in gates:
            if gate not in ['X', 'Y', 'Z', 'Had', 'S', 'T', 'CNOT', 'CNOT_10', 'SWAP', 'CZ',
                            'Rotation_x', 'Rotation_y', 'Rotation_z', 'Phase', 'Cphase']:
                print("Unknown gate:", gate)
        gate_functions = gates_lookup(gates)
        if noise_dropdown.value != "None":
            noise_dict[noise_dropdown.value.lower()] = noise_slider.value
        rho.apply_sequence_noise(gate_functions, [t for t in targets], noise_channels=noise_dict)
        global last_bloch_fig
        last_bloch_fig = plot_bloch_sphere(rho)
    with out1:
        global last_circuit_fig
        last_circuit_fig = plot_circuit(rho)
    with out2:
        global last_measurement_fig
        last_measurement_fig = plot_measure_DMs(rho, meas_shots=1000)
            
def on_save_figure(_):
    folder = "saved figures"
    os.makedirs(folder, exist_ok=True)

    date = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    prefix = filename_box.value + "_" + date
    saved_text = []
    if last_bloch_fig:
        last_bloch_fig.savefig(os.path.join(folder,prefix + "_bloch.png"), dpi=200, bbox_inches="tight", pad_inches=0.2)
        saved_text.append("Bloch spheres")
    if last_circuit_fig:
        last_circuit_fig.savefig(os.path.join(folder,prefix + "_circuit.png"), dpi=200, bbox_inches="tight", pad_inches=0.2)
        saved_text.append("Circuit diagram")
    if last_measurement_fig:
        last_measurement_fig.savefig(os.path.join(folder, prefix + "_measure.png"), dpi=200, bbox_inches="tight", pad_inches=0.2)
        saved_text.append("Measurement statistics")
    if len(saved_text) > 0:
        print("Saved: " + ", ".join(saved_text) + f" with prefix {prefix} in folder 'saved figures'.")
    else:
        print("No figures to save.")

def on_shutdown(_):
    ppid = os.getppid()  # Voilà server PID
    try:
        os.kill(ppid, signal.SIGTERM)  # stop Voilà server (ends terminal subprocess)
    except Exception:
        pass
    try:
        # also stop the kernel gracefully
        get_ipython().kernel.do_shutdown(restart=False)
    except Exception:
        pass
    # last-resort hard exit of the kernel
    os._exit(0)

# Map buttons to handlers
exit_btn.on_click(on_shutdown)
reinit_btn.on_click(on_initialise)
send_btn.on_click(on_send)
save_button.on_click(on_save_figure)

# Assign grid areas
title.layout.grid_area = 'title'
init_label.layout.grid_area = 'init_label'
reinit_btn.layout.grid_area = 'init_btn'
gate_label.layout.grid_area = 'gate_label'
text_in.layout.grid_area = 'text_in'
send_btn.layout.grid_area = 'send_btn'
noise_dropdown.layout.grid_area = 'noise_dropdown'
noise_slider.layout.grid_area = 'noise_slider'
save_label.layout.grid_area = 'save_label'
filename_box.layout.grid_area = 'filename_box'
save_button.layout.grid_area = 'save_btn'
exit_btn.layout.grid_area = 'exit_btn'
state_in.layout.grid_area = 'state_in'
noise_label.layout.grid_area = 'noise_label'


out2.layout = Layout(
    grid_area="out2",
    width="85%",
    height="75%",       
    border="1px solid #ccc",
    overflow="hidden",
    align_self="center"    
)

out.layout = Layout(
    grid_area='out',
    width="95%",
    height="300px",
    border="1px solid #ccc",
    overflow="hidden"
)

out1.layout = Layout(
    grid_area='out1',
    width="95%",
    height="250px",
    border="1px solid #ccc",
    overflow="hidden"
)

title.layout = Layout(
    grid_area="title",
    justify_self="center"  
)

# GridBox layout
grid = GridBox(
    children=[
        title, init_label, reinit_btn, state_in, noise_label,
        text_in, send_btn, gate_label, 
        noise_dropdown, noise_slider,
        out, out1, out2, filename_box, save_button, save_label, exit_btn
    ],
    layout=Layout(
        width='100%',
        grid_template_columns='0.9fr 1fr 3fr',
        grid_template_rows='auto auto auto auto auto 40px 300px 15px 250px, auto',
        grid_template_areas='''
"exit_btn title title"
"state_in init_btn init_label"
"noise_dropdown noise_slider noise_label"
"text_in send_btn gate_label"
"filename_box save_btn save_label"
"out2 out out"
"out2 . ."
"out2 out1 out1"
'''
    )
)

display(grid)


GridBox(children=(HTML(value="<h2 style='color:blue;'>Cub<i>e</i><sup>it</sup> Quantum Circuit Simulator</h2>"…